# Серия тиков с разной ценой, количество тиков N, расстояние во времени меньше T миллисекунд

In [1]:
from simple.ohlc import debounce, npJoin
from simple.plotly import chartFigure
from simple.funcs import symlog
from simple.backtest import npBacktestMarket, getProfit
import numpy as np
from simple.numpy import rolling_apply

In [2]:
threshold = 2_000_000  # default resolution in microseconds
period = 30

In [3]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
T[-5:]

rec.array([('2022-05-08T05:59:59.886999', 910037, 2537.16, 394.),
           ('2022-05-08T05:59:59.886999', 910038, 2537.16, 394.),
           ('2022-05-08T05:59:59.886999', 910039, 2537.16, 210.),
           ('2022-05-08T05:59:59.897000', 910040, 2537.16,   2.),
           ('2022-05-08T05:59:59.909999', 910041, 2537.16,   2.)],
          dtype={'names':['DT','LocalDT','Price','Volume'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [4]:
Buy = T[T.Volume > 0]
Sell = T[T.Volume < 0]

In [5]:
C = debounce(T)
C[-5:]

rec.array([('2022-05-08T05:59:50.540000', 155086, 2537.145,       0,  2535., 2535.,  0.000e+00,  3,  3,  0),
           ('2022-05-08T05:59:50.611000', 155087, 2537.185,   71000,  2380., 2378., -2.000e+00,  6,  5,  1),
           ('2022-05-08T05:59:51.163000', 155088, 2537.105,   18000,   395.,  197., -1.980e+02,  2,  1,  1),
           ('2022-05-08T05:59:59.780999', 155089, 2536.985, 8617999, 16592., 9984., -6.608e+03, 67, 55, 12),
           ('1970-01-01T00:00:00.000000', 155090, 2537.145,  129000,  3880., 3880.,  0.000e+00, 10, 10,  0)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [6]:
len(T), len(C)

(907307, 155091)

In [7]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2022-05-08T05:59:42.346000', 155074, 2536.975,      0, -1.000e+02,   0.,  -100., 1, 0, 1),
           ('2022-05-08T05:59:42.346000', 155075, 2536.935,      0, -1.970e+03,   0., -1970., 3, 0, 3),
           ('2022-05-08T05:59:42.346000', 155076, 2536.905,      0, -5.081e+03,   0., -5081., 6, 0, 6),
           ('2022-05-08T05:59:42.745000', 155077, 2536.875, 363000, -5.649e+03,   0., -5649., 7, 0, 7),
           ('2022-05-08T05:59:51.163000', 155088, 2537.105,  18000, -1.000e+00, 197.,  -198., 2, 1, 1)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [8]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [9]:
K = npJoin(C.DT, S.DT)
len(K)

155091

In [11]:
# calculate sum of duration/volume by sliding window
def getSum(x):
    return np.sum(x).astype(int)


Duration = rolling_apply(getSum, period, C.Duration)
Volume = rolling_apply(getSum, period, C.Volume)
len(Duration)

155091

In [14]:
# trade signal
s = 0
Signal = np.zeros_like(Duration)
for i in range(len(SignalA)):
    if Duration[i] < threshold and Volume[i] > 0:
        s = 1
    elif Duration[i] < threshold and C.Volume[i] < 0:
        s = -1
    Signal[i] = s

SignalA[-10:]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [15]:
D = npBacktestMarket(S.DT[K], S.A[K], S.B[K], SignalA, 0)
D[-5:]

rec.array([(154314, '2022-05-08T05:55:22.171999', 2539.98999023, 2539.73999023, 154418, '2022-05-08T05:56:51.321000', 2538.19995117, 2538.27490234,  1.),
           (154420, '2022-05-08T05:56:51.355000', 2538.22998047, 2538.23999023, 154658, '2022-05-08T05:57:55.601000', 2535.87988281, 2535.79980469, -1.),
           (154662, '2022-05-08T05:57:55.633999', 2535.84008789, 2535.83496094, 154829, '2022-05-08T05:58:41.071999', 2534.20996094, 2534.27490234,  1.),
           (154829, '2022-05-08T05:58:41.071999', 2534.20996094, 2534.27490234, 154851, '2022-05-08T05:58:41.629999', 2534.84008789, 2534.64990234, -1.),
           (154851, '2022-05-08T05:58:41.629999', 2534.84008789, 2534.64990234, 155018, '2022-05-08T05:59:09.130999', 2537.0300293 , 2537.08007812,  1.)],
          dtype=[('X0', '<i8'), ('T0', '<M8[us]'), ('Price0', '<f8'), ('MidPrice0', '<f8'), ('X1', '<i8'), ('T1', '<M8[us]'), ('Price1', '<f8'), ('MidPrice1', '<f8'), ('Size', '<f8')])

In [16]:
Long = D[D.Size > 0]
Short = D[D.Size < 0]

P = getProfit(D)
P[-5:]

rec.array([(154418, '2022-05-08T05:56:51.321000', -1.79003906, -1.46508789, 2.53909497, -4.32913403),
           (154658, '2022-05-08T05:57:55.601000',  2.35009766,  2.44018555, 2.53705493, -0.18695728),
           (154829, '2022-05-08T05:58:41.071999', -1.63012695, -1.56005859, 2.53502502, -4.16515198),
           (154851, '2022-05-08T05:58:41.629999', -0.63012695, -0.375     , 2.53452502, -3.16465198),
           (155018, '2022-05-08T05:59:09.130999',  2.18994141,  2.43017578, 2.53593506, -0.34599365)],
          dtype=[('Index', '<i8'), ('DT', '<M8[us]'), ('RawPnL', '<f8'), ('MidPnL', '<f8'), ('Fee', '<f8'), ('Profit', '<f8')])

In [18]:
# indexed chart with bidask
fig = chartFigure(height=800, rows=4,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.25),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.25),
    Bid=dict(x=C.Index, y=S.B[K], color='green', opacity=0.2),
    Ask=dict(x=C.Index, y=S.A[K], color='red', opacity=0.2),

    # trades
    EnterLong=dict(x=Long.X0, y=Long.Price0, mode='markers',
                   marker=dict(symbol='triangle-up', size=12, line_color='darkgreen', line_width=1, color='green')),
    ExitLong=dict(x=Long.X1, y=Long.Price1, mode='markers',
                  marker=dict(symbol='x', size=10, line_color='darkgreen', line_width=1, color='green', opacity=0.4)),

    EnterShort=dict(x=Short.X0, y=Short.Price0, mode='markers',
                    marker=dict(symbol='triangle-down', size=12, line_color='darkred', line_width=1, color='red')),
    ExitShort=dict(x=Short.X1, y=Short.Price1, mode='markers',
                   marker=dict(symbol='x', size=10, line_color='darkred', line_width=1, color='red', opacity=0.4)),

    # Signal and duration indicators
    Signal=dict(x=C.Index, y=SignalA, color='magenta', row=3, shape='hv'),
    Duration=dict(x=C.Index, y=Duration, color='blue', row=4),

    # Profit
    RawPnL=dict(x=P.Index, y=P.RawPnL.cumsum(), secondary_y=True, width=4, color='gray', opacity=0.2, shape='hv', connectgaps=True),
    Profit=dict(x=P.Index, y=P.Profit.cumsum(), secondary_y=True, width=6, color='blue', opacity=0.4, shape='hv', connectgaps=True)
)

fig.add_bar(name='Buy', x=Buy.Index, y=Buy.Volume, row=2, col=1, marker_color='green')
fig.add_bar(name='Sell', x=Sell.Index, y=Sell.Volume, row=2, col=1, marker_color='red')
fig.add_hline(y=threshold, row=4, col=1, line_dash='dot', line_color='gray')

# styling
fig.update_xaxes(spikemode='across+marker', spikedash='dot', spikethickness=2, spikesnap='cursor')
fig.update_traces(xaxis='x4')
fig.update_layout(legend=dict(x=0, y=1.05, bordercolor="lightgray", borderwidth=1))

fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.25,
   …